# Tarea 3 - Inteligencia Artificial
### Alejandra González Fernández y Gilbert Rodríguez Mejias

# Chatbot
## Con Spacy

In [5]:
# pip install spacy
# python -m spacy download en_core_web_sm
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

nlp = spacy.load('en_core_web_lg')

# List of answers to common questions about Dungeons & Dragons
answersSpacy = [ "Dungeons & Dragons is a fantasy tabletop role-playing game.",
            "To create a character, you'll choose a race, class, abilities, and background.",
            "Classes are fighters, wizards, rogues, clerics, paladins, rangers, and more.",
            "Ability scores represent a character's innate abilities and strengths. They include Strength, Dexterity, Constitution, Intelligence, Wisdom, and Charisma.",
            "Combat in D&D is turn-based, with each character and creature taking actions in a specific order.",
            "Spells are magical abilities that characters can cast to produce various effects, such as dealing damage, healing allies, or manipulating the environment.",
            "Some famous D&D campaigns include 'Curse of Strahd,' 'Tomb of Annihilation,' 'Storm King's Thunder,' 'Dragon Heist,' and 'Out of the Abyss,' among others.",
            "The Dungeon Master (DM) is the player who acts as the game's storyteller and referee, guiding the players through the adventure, describing scenes, and controlling non-player characters (NPCs).",
            "Sorcerers derive their magical abilities from innate talent, wizards learn magic through study and practice, and warlocks make pacts with otherworldly beings for their magical powers.",
            "Alignment describes a character's moral and ethical outlook. The alignment help define a character's beliefs and motivations."]

# Generate embeddings for each answer
# embeddings = [nlp(answer.lower()).vector for answer in answers]
embeddingsSpacy = []
for answer in answersSpacy:
    embeddingsSpacy.append(np.array([t.vector for t in nlp(answer.lower())]).mean(0).reshape(1,-1)) # mean of word vectors

knowledge_datasetSpacy = []
for i in range(0, len(answersSpacy)):
    knowledge_datasetSpacy.append((answersSpacy[i], embeddingsSpacy[i])) # tuple of answer and its embedding


def get_userQuestionEmbeddingSpacy(user_question):
    return np.array([t.vector for t in nlp(user_question.lower())]).mean(0).reshape(1,-1) # mean of question embedding vectors

def get_most_similar_answerSpacy(user_question):
    user_question_embeddingSpacy = get_userQuestionEmbeddingSpacy(user_question)
    max_similaritySpacy = -1
    most_similar_answerSpacy = ""
    for answer, answer_embedding in knowledge_datasetSpacy:
        print(cosine_similarity(user_question_embeddingSpacy, answer_embedding))
        similarity = cosine_similarity(user_question_embeddingSpacy, answer_embedding) # cosine similarity

        if similarity > max_similaritySpacy:
            max_similaritySpacy = similarity
            most_similar_answerSpacy = answer

    return most_similar_answerSpacy

## Con Transformers modelo Bert

In [6]:
import torch
from transformers import BertModel, BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained BERT model and tokenizer
model = BertModel.from_pretrained('bert-large-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# List of answers to common questions about Dungeons & Dragons
answers = [ "Dungeons & Dragons is a fantasy tabletop role-playing game.",
            "To create a character, you'll choose a race, class, abilities, and background.",
            "Classes are fighters, wizards, rogues, clerics, paladins, rangers, and more.",
            "Ability scores represent a character's innate abilities and strengths. They include Strength, Dexterity, Constitution, Intelligence, Wisdom, and Charisma.",
            "Combat in D&D is turn-based, with each character and creature taking actions in a specific order.",
            "Spells are magical abilities that characters can cast to produce various effects, such as dealing damage, healing allies, or manipulating the environment.",
            "Some famous D&D campaigns include 'Curse of Strahd,' 'Tomb of Annihilation,' 'Storm King's Thunder,' 'Dragon Heist,' and 'Out of the Abyss,' among others.",
            "The Dungeon Master (DM) is the player who acts as the game's storyteller and referee, guiding the players through the adventure, describing scenes, and controlling non-player characters (NPCs).",
            "Sorcerers derive their magical abilities from innate talent, wizards learn magic through study and practice, and warlocks make pacts with otherworldly beings for their magical powers.",
            "Alignment describes a character's moral and ethical outlook. The alignment help define a character's beliefs and motivations."]

# Generate embeddings for each answer
# embeddings = [nlp(answer.lower()).vector for answer in answers]
answers_tokens = [tokenizer.encode(answer.lower(), return_tensors='pt') for answer in answers]

with torch.no_grad():
    embeddings = [model(answers_token)[0][:, 0, :].numpy() for answers_token in answers_tokens]

knowledge_dataset = []
for i in range(0, len(answers)):
    knowledge_dataset.append((answers[i], embeddings[i])) # tuple of answer and its embedding


def get_userQuestionEmbedding(user_question):
    question_tokens = tokenizer.encode(user_question.lower(), return_tensors='pt')
    with torch.no_grad():
        question_embedding = model(question_tokens)[0][:, 0, :].numpy()  # Take the [CLS] token embedding for the question
    return question_embedding

def get_most_similar_answer(user_question):
    user_question_embedding = get_userQuestionEmbedding(user_question)
    max_similarity = -1
    most_similar_answer = ""
    for answer, answer_embedding in knowledge_dataset:
        print(cosine_similarity(user_question_embedding, answer_embedding))
        similarity = cosine_similarity(user_question_embedding, answer_embedding) # cosine similarity

        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_answer = answer

    return most_similar_answer

d:\Tec\IA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Posibles preguntas

* What is Dungeons & Dragons?
* How do I create a character?
* What are the different classes?
* What are ability scores?
* How does combat work?
* What are spells?
* What are some famous campaigns or adventures?
* What is a Dungeon Master (DM)?
* Can you explain the difference between a sorcerer, wizard, and warlock?
* How alignment work?

In [19]:
user_question = "Can you explain the difference between a sorcerer, wizard, and warlock?"
print("Usin Spacy")
answerSpacy = get_most_similar_answerSpacy(user_question)
print(answerSpacy)

print("Using BERT")
answer = get_most_similar_answer(user_question)
print(answer)

Usin Spacy
[[0.5876517]]
[[0.9241441]]
[[0.7811913]]
[[0.8717209]]
[[0.82002825]]
[[0.8646748]]
[[0.5361411]]
[[0.71873486]]
[[0.84280837]]
[[0.84880126]]
To create a character, you'll choose a race, class, abilities, and background.
Using BERT
[[0.7186961]]
[[0.7812069]]
[[0.7476671]]
[[0.7673046]]
[[0.7575501]]
[[0.7672875]]
[[0.7755226]]
[[0.74145865]]
[[0.79439735]]
[[0.7583243]]
Sorcerers derive their magical abilities from innate talent, wizards learn magic through study and practice, and warlocks make pacts with otherworldly beings for their magical powers.
